# TFEX Trading Statistics Service

Learn how to fetch trading statistics including settlement prices, margin requirements, and days to maturity for TFEX futures and options contracts.

## Overview

### What are Trading Statistics?

Trading statistics provide essential real-time data for each TFEX contract:
- **Settlement Price** - Official closing price used for marking positions
- **Margin Requirements** - Capital needed to trade (Initial & Maintenance)
- **Days to Maturity** - Time remaining until contract expiration
- **Theoretical Price** - Fair value based on pricing models

### Key Derivatives Concepts

**Settlement Price:**
- Official price calculated at market close
- Used to determine daily profit/loss on positions
- Affects margin requirements
- Final settlement determines contract value at expiration

**Margin Requirements:**
- **Initial Margin (IM)** - Minimum deposit to open a position
- **Maintenance Margin (MM)** - Minimum to keep position open
- If account falls below MM, you get a **margin call** (must add funds or close position)
- Margin = Deposit, not the full contract value (leverage!)

**Days to Maturity:**
- Critical for contract rollover planning
- Options lose value as expiration approaches ("time decay")
- Less liquidity near expiration

**Theoretical Price:**
- Fair value calculated using pricing models
- Compare with settlement to find mispricing
- Important for options trading strategies

### What This Service Does

The TFEX Trading Statistics Service allows you to:
- **Check margin requirements** before trading
- **Calculate position sizing** based on available capital
- **Monitor settlement prices** for mark-to-market calculations
- **Track days to maturity** for rollover planning
- **Compare theoretical vs actual prices** for arbitrage opportunities

### When to Use This Service

Use this service when you need to:
- Plan how many contracts you can trade with your capital
- Calculate daily profit/loss on existing positions
- Monitor margin requirements to avoid margin calls
- Plan contract rollovers based on expiration dates
- Analyze price efficiency for trading opportunities

## Prerequisites

### Installation

You need to install the `settfex` package:

```bash
pip install settfex
```

### Required Imports

This notebook uses:
- `settfex` - The main library for TFEX data
- `asyncio` - For asynchronous programming
- `datetime` - For working with dates

In [1]:
# Install settfex (uncomment to run)
# !pip install settfex

In [2]:
# Import required libraries
import asyncio
from datetime import datetime

# Import TFEX services
from settfex.services.tfex import get_trading_statistics, get_series_list

## Basic Usage

Let's start by fetching trading statistics for a single TFEX contract.

In [3]:
# Fetch trading statistics for S50Z25 (SET50 Futures December 2025)
# Note: Jupyter notebooks handle async automatically in most modern versions
stats = await get_trading_statistics("S50Z25")

# Display basic information
print(f"Symbol: {stats.symbol}")
print(f"Market Time: {stats.market_time}")
print(f"Last Trading Date: {stats.last_trading_date}")
print(f"Days to Maturity: {stats.day_to_maturity}")
print(f"Is Options: {stats.is_options}")

# Expected output:
# Symbol: S50Z25
# Market Time: 2025-10-05 15:30:00+07:00
# Last Trading Date: 2025-12-29 16:30:00+07:00
# Days to Maturity: 85
# Is Options: False

2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:__init__:77 | TradingStatisticsService initialized with base_url=https://www.tfex.co.th
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:119 | Fetching TFEX trading statistics for S50Z25 from https://www.tfex.co.th/api/set/tfex/series/S50Z25/trading-statistics
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 17:41:21 | INFO     | settfex.utils.session_manager:__init__:98 | SessionManager created with browser=chrome120, warmup_site=tfex, cache=enabled
2025-10-05 17:41:21 | INFO     | settfex.utils.session_cache:__init__:76 | SessionCache initialized at /Users/sarat/.settfex/cache (ttl=3600s, size_limit=100MB)
2025-10-05 17:41:21 | SUCCESS  | settfex.utils.session_manager:_try_load_from_cache:202 | ✓ Loaded session from cache: 4 cookies (age=9s, bro

Symbol: S50Z25
Market Time: 2025-10-04 03:05:08.070566+07:00
Last Trading Date: 2025-12-29 16:30:00+07:00
Days to Maturity: 86
Is Options: False


### Understanding Settlement Prices

Settlement prices are crucial for calculating your daily profit/loss.

In [4]:
# Display pricing information
print("PRICING INFORMATION\n")
print(f"Settlement Price: {stats.settlement_price:.5f}")
print(f"Prior Settlement: {stats.prior_settlement_price:.5f}")

# Calculate daily change
price_change = stats.settlement_price - stats.prior_settlement_price
percent_change = (price_change / stats.prior_settlement_price) * 100

print(f"\nDaily Change: {price_change:+.5f} ({percent_change:+.2f}%)")

# Show theoretical price if available
if stats.has_theoretical_price:
    print(f"\nTheoretical Price: {stats.theoretical_price:.5f}")
    diff = stats.settlement_price - stats.theoretical_price
    print(f"Difference from Theoretical: {diff:+.5f}")

# Expected output:
# PRICING INFORMATION
#
# Settlement Price: 830.30000
# Prior Settlement: 825.90000
#
# Daily Change: +4.40000 (+0.53%)
#
# Theoretical Price: 831.87760
# Difference from Theoretical: -1.57760

PRICING INFORMATION

Settlement Price: 830.30000
Prior Settlement: 825.90000

Daily Change: +4.40000 (+0.53%)

Theoretical Price: 831.87760
Difference from Theoretical: -1.57760


### Understanding Margin Requirements

**What is Margin?**

Margin is a deposit (not a fee!) required to trade derivatives:
- **Initial Margin (IM)** - Amount needed to open a position
- **Maintenance Margin (MM)** - Minimum to keep position open

**Example:** If IM = 10,000 THB and MM = 7,000 THB:
- You need 10,000 THB to open a position
- If losses reduce your account to 6,999 THB, you get a margin call
- You must add funds to bring account back above 7,000 THB
- If you can't, your broker will close your position

In [5]:
# Display margin requirements
print("MARGIN REQUIREMENTS\n")
print(f"Initial Margin (IM): {stats.im:,.2f} THB")
print(f"Maintenance Margin (MM): {stats.mm:,.2f} THB")

# Calculate margin ratio
margin_ratio = (stats.mm / stats.im) * 100
margin_buffer = stats.im - stats.mm

print(f"\nMargin Ratio (MM/IM): {margin_ratio:.2f}%")
print(f"Margin Buffer: {margin_buffer:,.2f} THB ({100 - margin_ratio:.2f}%)")

# Explain what this means
print("\nWhat this means:")
print(f"  • You need {stats.im:,.0f} THB to open 1 contract")
print(f"  • Account can drop to {stats.mm:,.0f} THB before margin call")
print(f"  • You have {margin_buffer:,.0f} THB cushion for losses")

# Expected output:
# MARGIN REQUIREMENTS
#
# Initial Margin (IM): 10,080.00 THB
# Maintenance Margin (MM): 7,084.80 THB
#
# Margin Ratio (MM/IM): 70.28%
# Margin Buffer: 2,995.20 THB (29.72%)
#
# What this means:
#   • You need 10,080 THB to open 1 contract
#   • Account can drop to 7,085 THB before margin call
#   • You have 2,995 THB cushion for losses

MARGIN REQUIREMENTS

Initial Margin (IM): 10,080.00 THB
Maintenance Margin (MM): 7,084.80 THB

Margin Ratio (MM/IM): 70.29%
Margin Buffer: 2,995.20 THB (29.71%)

What this means:
  • You need 10,080 THB to open 1 contract
  • Account can drop to 7,085 THB before margin call
  • You have 2,995 THB cushion for losses


## Advanced Usage - Multiple Contracts

Let's compare statistics across multiple TFEX series.

In [6]:
# Compare multiple SET50 futures contracts
symbols = ["S50V25", "S50X25", "S50Z25"]  # October, November, December 2025

print("COMPARING SET50 FUTURES CONTRACTS\n")
print(f"{'Symbol':<12} {'Settlement':<15} {'Days Left':<12} {'IM':<15} {'MM':<15}")
print("-" * 80)

for symbol in symbols:
    try:
        stats = await get_trading_statistics(symbol)
        print(
            f"{stats.symbol:<12} "
            f"{stats.settlement_price:<15.5f} "
            f"{stats.day_to_maturity:<12} "
            f"{stats.im:<15,.2f} "
            f"{stats.mm:<15,.2f}"
        )
    except Exception as e:
        print(f"{symbol:<12} Error: {e}")

# Expected output:
# COMPARING SET50 FUTURES CONTRACTS
#
# Symbol       Settlement      Days Left    IM              MM             
# --------------------------------------------------------------------------------
# S50V25       828.50000       25           10,080.00       7,084.80      
# S50X25       829.75000       55           10,080.00       7,084.80      
# S50Z25       830.30000       85           10,080.00       7,084.80

2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:__init__:77 | TradingStatisticsService initialized with base_url=https://www.tfex.co.th
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:119 | Fetching TFEX trading statistics for S50V25 from https://www.tfex.co.th/api/set/tfex/series/S50V25/trading-statistics
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:fetch:303 | Fetch successful: url=https://www.tfex.co.th/api/set/tfex/series/S50V25/trading-statistics, status=200, elapsed=0.01s, size=320 bytes
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:148 | Successfully fetched trading statistics for S50V25: settlement_price=834.0, day_to_maturity=26, im=10080.0
2025-10-05 17:41:21 | INFO     | settfex

COMPARING SET50 FUTURES CONTRACTS

Symbol       Settlement      Days Left    IM              MM             
--------------------------------------------------------------------------------
S50V25       834.00000       26           10,080.00       7,084.80       
S50X25       831.30000       54           10,080.00       7,084.80       
S50Z25       830.30000       86           10,080.00       7,084.80       


## Derivatives Trading Use Cases

Now let's explore real-world scenarios for derivatives traders.

### Use Case 1: Position Sizing Based on Margin

**The Problem:**
You have 500,000 THB capital. How many SET50 futures contracts can you trade?

**The Calculation:**
- Each contract requires Initial Margin (IM)
- You should also consider risk management (don't use 100% of capital!)
- Leave buffer for potential losses to avoid margin calls

Let's calculate the optimal position size:

In [7]:
# Position sizing calculator
async def calculate_position_size(symbol, capital, risk_percentage=80):
    """
    Calculate maximum position size based on available capital.
    
    Args:
        symbol: TFEX contract symbol
        capital: Available trading capital (THB)
        risk_percentage: Max % of capital to use (default 80%)
    """
    stats = await get_trading_statistics(symbol)
    
    # Calculate usable capital (leaving safety buffer)
    usable_capital = capital * (risk_percentage / 100)
    
    # Maximum contracts based on IM
    max_contracts = int(usable_capital / stats.im)
    
    # Calculate actual margin usage
    total_im = max_contracts * stats.im
    total_mm = max_contracts * stats.mm
    
    print(f"POSITION SIZING FOR {symbol}\n")
    print(f"Available Capital: {capital:,.2f} THB")
    print(f"Usable Capital ({risk_percentage}%): {usable_capital:,.2f} THB")
    print(f"\nMargin per Contract:")
    print(f"  Initial Margin (IM): {stats.im:,.2f} THB")
    print(f"  Maintenance Margin (MM): {stats.mm:,.2f} THB")
    
    print(f"\nRECOMMENDED POSITION:")
    print(f"  Maximum Contracts: {max_contracts}")
    print(f"  Total IM Required: {total_im:,.2f} THB")
    print(f"  Total MM Required: {total_mm:,.2f} THB")
    print(f"  Remaining Capital: {capital - total_im:,.2f} THB")
    
    # Risk analysis
    buffer = total_im - total_mm
    print(f"\nRISK ANALYSIS:")
    print(f"  Margin Buffer: {buffer:,.2f} THB")
    print(f"  Max Loss Before Margin Call: {buffer:,.2f} THB")
    print(f"  Loss per Point: {max_contracts * 200:.2f} THB")  # SET50 multiplier = 200
    
    # Calculate points cushion
    points_cushion = buffer / (max_contracts * 200) if max_contracts > 0 else 0
    print(f"  Price Can Drop: {points_cushion:.2f} points before margin call")
    
    return max_contracts

# Calculate for 500,000 THB capital
contracts = await calculate_position_size("S50Z25", 500000, risk_percentage=80)

# Expected output:
# POSITION SIZING FOR S50Z25
#
# Available Capital: 500,000.00 THB
# Usable Capital (80%): 400,000.00 THB
#
# Margin per Contract:
#   Initial Margin (IM): 10,080.00 THB
#   Maintenance Margin (MM): 7,084.80 THB
#
# RECOMMENDED POSITION:
#   Maximum Contracts: 39
#   Total IM Required: 393,120.00 THB
#   Total MM Required: 276,307.20 THB
#   Remaining Capital: 106,880.00 THB
#
# RISK ANALYSIS:
#   Margin Buffer: 116,812.80 THB
#   Max Loss Before Margin Call: 116,812.80 THB
#   Loss per Point: 7,800.00 THB
#   Price Can Drop: 14.98 points before margin call

2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:__init__:77 | TradingStatisticsService initialized with base_url=https://www.tfex.co.th
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:119 | Fetching TFEX trading statistics for S50Z25 from https://www.tfex.co.th/api/set/tfex/series/S50Z25/trading-statistics
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:fetch:303 | Fetch successful: url=https://www.tfex.co.th/api/set/tfex/series/S50Z25/trading-statistics, status=200, elapsed=0.01s, size=320 bytes
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:148 | Successfully fetched trading statistics for S50Z25: settlement_price=830.3, day_to_maturity=86, im=10080.0


POSITION SIZING FOR S50Z25

Available Capital: 500,000.00 THB
Usable Capital (80%): 400,000.00 THB

Margin per Contract:
  Initial Margin (IM): 10,080.00 THB
  Maintenance Margin (MM): 7,084.80 THB

RECOMMENDED POSITION:
  Maximum Contracts: 39
  Total IM Required: 393,120.00 THB
  Total MM Required: 276,307.20 THB
  Remaining Capital: 106,880.00 THB

RISK ANALYSIS:
  Margin Buffer: 116,812.80 THB
  Max Loss Before Margin Call: 116,812.80 THB
  Loss per Point: 7800.00 THB
  Price Can Drop: 14.98 points before margin call


### Use Case 2: Daily Profit/Loss Calculation

**The Problem:**
You hold 10 SET50 futures contracts. How much did you gain/lose today?

**The Calculation:**
- Daily P/L = (Today's Settlement - Prior Settlement) × Multiplier × Contracts
- SET50 futures multiplier = 200 THB per index point
- This is marked-to-market daily (realized in your account)

Let's calculate daily profit/loss:

In [8]:
# Daily P/L calculator
async def calculate_daily_pnl(symbol, contracts, position_type="LONG"):
    """
    Calculate daily profit/loss on existing futures position.
    
    Args:
        symbol: TFEX contract symbol
        contracts: Number of contracts held
        position_type: "LONG" or "SHORT"
    """
    stats = await get_trading_statistics(symbol)
    
    # Calculate price change
    price_change = stats.settlement_price - stats.prior_settlement_price
    percent_change = (price_change / stats.prior_settlement_price) * 100
    
    # SET50 multiplier = 200 THB per point
    multiplier = 200
    
    # Calculate P/L (positive for long if price up, negative if price down)
    if position_type.upper() == "LONG":
        pnl = price_change * multiplier * contracts
    else:  # SHORT
        pnl = -price_change * multiplier * contracts
    
    print(f"DAILY P/L CALCULATION - {symbol}\n")
    print(f"Position: {position_type.upper()} {contracts} contracts")
    print(f"\nPricing:")
    print(f"  Prior Settlement: {stats.prior_settlement_price:.5f}")
    print(f"  Current Settlement: {stats.settlement_price:.5f}")
    print(f"  Price Change: {price_change:+.5f} ({percent_change:+.2f}%)")
    
    print(f"\nP/L Calculation:")
    print(f"  Price Change: {price_change:.5f} points")
    print(f"  Multiplier: {multiplier} THB/point")
    print(f"  Contracts: {contracts}")
    print(f"  Formula: {price_change:.5f} × {multiplier} × {contracts}")
    
    print(f"\nDAILY P/L: {pnl:+,.2f} THB")
    
    # Show cumulative margin impact
    total_margin = contracts * stats.im
    pnl_percentage = (pnl / total_margin) * 100
    
    print(f"\nMargin Impact:")
    print(f"  Total Margin: {total_margin:,.2f} THB")
    print(f"  P/L vs Margin: {pnl_percentage:+.2f}%")
    
    return pnl

# Example: Long 10 contracts
pnl = await calculate_daily_pnl("S50Z25", 10, position_type="LONG")

# Expected output:
# DAILY P/L CALCULATION - S50Z25
#
# Position: LONG 10 contracts
#
# Pricing:
#   Prior Settlement: 825.90000
#   Current Settlement: 830.30000
#   Price Change: +4.40000 (+0.53%)
#
# P/L Calculation:
#   Price Change: 4.40000 points
#   Multiplier: 200 THB/point
#   Contracts: 10
#   Formula: 4.40000 × 200 × 10
#
# DAILY P/L: +8,800.00 THB
#
# Margin Impact:
#   Total Margin: 100,800.00 THB
#   P/L vs Margin: +8.73%

2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:__init__:77 | TradingStatisticsService initialized with base_url=https://www.tfex.co.th
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:119 | Fetching TFEX trading statistics for S50Z25 from https://www.tfex.co.th/api/set/tfex/series/S50Z25/trading-statistics
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:fetch:303 | Fetch successful: url=https://www.tfex.co.th/api/set/tfex/series/S50Z25/trading-statistics, status=200, elapsed=0.02s, size=320 bytes
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:148 | Successfully fetched trading statistics for S50Z25: settlement_price=830.3, day_to_maturity=86, im=10080.0


DAILY P/L CALCULATION - S50Z25

Position: LONG 10 contracts

Pricing:
  Prior Settlement: 825.90000
  Current Settlement: 830.30000
  Price Change: +4.40000 (+0.53%)

P/L Calculation:
  Price Change: 4.40000 points
  Multiplier: 200 THB/point
  Contracts: 10
  Formula: 4.40000 × 200 × 10

DAILY P/L: +8,800.00 THB

Margin Impact:
  Total Margin: 100,800.00 THB
  P/L vs Margin: +8.73%


In [9]:
# Compare LONG vs SHORT position
print("=" * 80)
print("COMPARING LONG VS SHORT POSITIONS")
print("=" * 80)

print("\n[LONG 10 CONTRACTS]")
long_pnl = await calculate_daily_pnl("S50Z25", 10, position_type="LONG")

print("\n" + "=" * 80)
print("\n[SHORT 10 CONTRACTS]")
short_pnl = await calculate_daily_pnl("S50Z25", 10, position_type="SHORT")

print("\n" + "=" * 80)
print(f"\nSUMMARY:")
print(f"  LONG P/L: {long_pnl:+,.2f} THB")
print(f"  SHORT P/L: {short_pnl:+,.2f} THB")
print(f"  Difference: {abs(long_pnl - short_pnl):,.2f} THB")

# Expected output shows opposite P/L for long vs short

2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:__init__:77 | TradingStatisticsService initialized with base_url=https://www.tfex.co.th
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:119 | Fetching TFEX trading statistics for S50Z25 from https://www.tfex.co.th/api/set/tfex/series/S50Z25/trading-statistics
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:fetch:303 | Fetch successful: url=https://www.tfex.co.th/api/set/tfex/series/S50Z25/trading-statistics, status=200, elapsed=0.01s, size=320 bytes
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:148 | Successfully fetched trading statistics for S50Z25: settlement_price=830.3, day_to_maturity=86, im=10080.0
2025-10-05 17:41:21 | INFO     | settfex

COMPARING LONG VS SHORT POSITIONS

[LONG 10 CONTRACTS]
DAILY P/L CALCULATION - S50Z25

Position: LONG 10 contracts

Pricing:
  Prior Settlement: 825.90000
  Current Settlement: 830.30000
  Price Change: +4.40000 (+0.53%)

P/L Calculation:
  Price Change: 4.40000 points
  Multiplier: 200 THB/point
  Contracts: 10
  Formula: 4.40000 × 200 × 10

DAILY P/L: +8,800.00 THB

Margin Impact:
  Total Margin: 100,800.00 THB
  P/L vs Margin: +8.73%


[SHORT 10 CONTRACTS]
DAILY P/L CALCULATION - S50Z25

Position: SHORT 10 contracts

Pricing:
  Prior Settlement: 825.90000
  Current Settlement: 830.30000
  Price Change: +4.40000 (+0.53%)

P/L Calculation:
  Price Change: 4.40000 points
  Multiplier: 200 THB/point
  Contracts: 10
  Formula: 4.40000 × 200 × 10

DAILY P/L: -8,800.00 THB

Margin Impact:
  Total Margin: 100,800.00 THB
  P/L vs Margin: -8.73%


SUMMARY:
  LONG P/L: +8,800.00 THB
  SHORT P/L: -8,800.00 THB
  Difference: 17,600.00 THB


### Use Case 3: Contract Rollover Timing

**The Problem:**
Your futures contract is expiring soon. When should you roll to the next month?

**Factors to Consider:**
- **Days to maturity** - Avoid last few days (low liquidity)
- **Roll cost** - Price difference between contracts
- **Volume** - Ensure sufficient liquidity for execution

**Common Practice:**
- Professional traders roll 5-10 days before expiration
- Retail traders often wait until last week (higher risk)

Let's create a rollover alert system:

In [10]:
# Contract rollover analyzer
async def analyze_rollover_timing(front_month_symbol, next_month_symbol):
    """
    Analyze rollover timing and cost between two contract months.
    
    Args:
        front_month_symbol: Current contract symbol
        next_month_symbol: Next contract symbol
    """
    # Fetch statistics for both contracts
    front_stats = await get_trading_statistics(front_month_symbol)
    next_stats = await get_trading_statistics(next_month_symbol)
    
    print(f"ROLLOVER ANALYSIS\n")
    print(f"Current Position: {front_stats.symbol}")
    print(f"Roll Target: {next_stats.symbol}")
    
    # Timing analysis
    print(f"\nTIMING:")
    print(f"  {front_stats.symbol} expires in: {front_stats.day_to_maturity} days")
    print(f"  {front_stats.symbol} expiry: {front_stats.last_trading_date.date()}")
    print(f"  {next_stats.symbol} expiry: {next_stats.last_trading_date.date()}")
    
    # Rollover urgency
    days_left = front_stats.day_to_maturity
    print(f"\nURGENCY:")
    if days_left <= 5:
        urgency = "🔴 CRITICAL"
        recommendation = "Roll IMMEDIATELY! Very low liquidity expected."
    elif days_left <= 10:
        urgency = "🟡 HIGH"
        recommendation = "Roll in next 1-2 days. Optimal timing window."
    elif days_left <= 15:
        urgency = "🟢 MODERATE"
        recommendation = "Plan rollover this week. Good liquidity expected."
    else:
        urgency = "⚪ LOW"
        recommendation = "Monitor daily. No rush yet."
    
    print(f"  Level: {urgency}")
    print(f"  Recommendation: {recommendation}")
    
    # Roll cost analysis
    roll_cost = next_stats.settlement_price - front_stats.settlement_price
    roll_cost_percent = (roll_cost / front_stats.settlement_price) * 100
    multiplier = 200  # SET50 multiplier
    
    print(f"\nROLL COST (per contract):")
    print(f"  {front_stats.symbol} settlement: {front_stats.settlement_price:.5f}")
    print(f"  {next_stats.symbol} settlement: {next_stats.settlement_price:.5f}")
    print(f"  Price difference: {roll_cost:+.5f} points ({roll_cost_percent:+.2f}%)")
    print(f"  Cost per contract: {roll_cost * multiplier:+,.2f} THB")
    
    # Calculate total cost for example position
    example_contracts = 10
    total_roll_cost = roll_cost * multiplier * example_contracts
    
    print(f"\nEXAMPLE: Rolling {example_contracts} contracts")
    print(f"  Total roll cost: {total_roll_cost:+,.2f} THB")
    
    # Contango vs Backwardation
    if roll_cost > 0:
        market_structure = "Contango (next month more expensive)"
        impact = "Cost to maintain long position"
    elif roll_cost < 0:
        market_structure = "Backwardation (next month cheaper)"
        impact = "Gain to maintain long position"
    else:
        market_structure = "Flat (same price)"
        impact = "No roll cost"
    
    print(f"\nMARKET STRUCTURE:")
    print(f"  {market_structure}")
    print(f"  Impact: {impact}")
    
    return roll_cost

# Analyze rollover from October to November
roll_cost = await analyze_rollover_timing("S50V25", "S50X25")

# Expected output:
# ROLLOVER ANALYSIS
#
# Current Position: S50V25
# Roll Target: S50X25
#
# TIMING:
#   S50V25 expires in: 25 days
#   S50V25 expiry: 2025-10-30
#   S50X25 expiry: 2025-11-27
#
# URGENCY:
#   Level: 🟢 MODERATE
#   Recommendation: Plan rollover this week. Good liquidity expected.
#
# ROLL COST (per contract):
#   S50V25 settlement: 828.50000
#   S50X25 settlement: 829.75000
#   Price difference: +1.25000 points (+0.15%)
#   Cost per contract: +250.00 THB
#
# EXAMPLE: Rolling 10 contracts
#   Total roll cost: +2,500.00 THB

2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:__init__:77 | TradingStatisticsService initialized with base_url=https://www.tfex.co.th
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:119 | Fetching TFEX trading statistics for S50V25 from https://www.tfex.co.th/api/set/tfex/series/S50V25/trading-statistics
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:fetch:303 | Fetch successful: url=https://www.tfex.co.th/api/set/tfex/series/S50V25/trading-statistics, status=200, elapsed=0.01s, size=320 bytes
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:148 | Successfully fetched trading statistics for S50V25: settlement_price=834.0, day_to_maturity=26, im=10080.0
2025-10-05 17:41:21 | INFO     | settfex

ROLLOVER ANALYSIS

Current Position: S50V25
Roll Target: S50X25

TIMING:
  S50V25 expires in: 26 days
  S50V25 expiry: 2025-10-30
  S50X25 expiry: 2025-11-27

URGENCY:
  Level: ⚪ LOW
  Recommendation: Monitor daily. No rush yet.

ROLL COST (per contract):
  S50V25 settlement: 834.00000
  S50X25 settlement: 831.30000
  Price difference: -2.70000 points (-0.32%)
  Cost per contract: -540.00 THB

EXAMPLE: Rolling 10 contracts
  Total roll cost: -5,400.00 THB

MARKET STRUCTURE:
  Backwardation (next month cheaper)
  Impact: Gain to maintain long position


### Use Case 4: Margin Call Risk Monitor

**What is a Margin Call?**

A margin call occurs when your account equity falls below the maintenance margin:
- Broker requires you to deposit more funds
- If you can't, broker will close your position (forced liquidation)
- Usually happens in losing trades or high volatility

**How to Avoid:**
- Monitor your margin usage daily
- Keep buffer above maintenance margin
- Reduce position size if getting close to MM

Let's create a margin call risk monitor:

In [11]:
# Margin call risk monitor
async def monitor_margin_call_risk(symbol, contracts, current_equity):
    """
    Monitor margin call risk for existing position.
    
    Args:
        symbol: TFEX contract symbol
        contracts: Number of contracts held
        current_equity: Current account equity (THB)
    """
    stats = await get_trading_statistics(symbol)
    
    # Calculate margin requirements
    total_im = contracts * stats.im
    total_mm = contracts * stats.mm
    
    # Calculate equity buffer
    equity_buffer = current_equity - total_mm
    equity_above_im = current_equity - total_im
    
    # Calculate margin usage
    margin_usage = (total_im / current_equity) * 100
    equity_vs_mm = (current_equity / total_mm) * 100
    
    print(f"MARGIN CALL RISK MONITOR - {symbol}\n")
    print(f"Position: {contracts} contracts")
    print(f"Current Equity: {current_equity:,.2f} THB")
    
    print(f"\nMARGIN REQUIREMENTS:")
    print(f"  Initial Margin (IM): {total_im:,.2f} THB")
    print(f"  Maintenance Margin (MM): {total_mm:,.2f} THB")
    print(f"  Margin Usage: {margin_usage:.2f}%")
    
    print(f"\nEQUITY ANALYSIS:")
    print(f"  Equity above IM: {equity_above_im:+,.2f} THB")
    print(f"  Equity above MM: {equity_buffer:+,.2f} THB")
    print(f"  Equity vs MM: {equity_vs_mm:.2f}%")
    
    # Risk assessment
    print(f"\nRISK ASSESSMENT:")
    
    if current_equity < total_mm:
        risk_level = "🔴 MARGIN CALL"
        action = "URGENT: Add funds immediately or position will be liquidated!"
    elif current_equity < total_im:
        risk_level = "🟠 CRITICAL"
        deficit = total_im - current_equity
        action = f"Add {deficit:,.2f} THB to avoid margin call"
    elif equity_vs_mm < 110:
        risk_level = "🟡 HIGH"
        action = "Consider adding funds or reducing position size"
    elif equity_vs_mm < 120:
        risk_level = "🟢 MODERATE"
        action = "Monitor closely. Small buffer remaining."
    else:
        risk_level = "✅ LOW"
        action = "Healthy margin buffer. No action needed."
    
    print(f"  Risk Level: {risk_level}")
    print(f"  Action: {action}")
    
    # Calculate how much account can lose
    if equity_buffer > 0:
        multiplier = 200
        max_loss_points = equity_buffer / (contracts * multiplier)
        max_loss_percent = (max_loss_points / stats.settlement_price) * 100
        
        print(f"\nMAX SUSTAINABLE LOSS:")
        print(f"  Maximum loss: {equity_buffer:,.2f} THB")
        print(f"  Points cushion: {max_loss_points:.2f} points")
        print(f"  Percentage cushion: {max_loss_percent:.2f}%")
        print(f"  Margin call if price drops below: {stats.settlement_price - max_loss_points:.2f}")

# Example 1: Healthy position
print("EXAMPLE 1: Healthy Position")
print("=" * 80)
await monitor_margin_call_risk("S50Z25", contracts=10, current_equity=200000)

print("\n\n")

# Example 2: Risky position
print("EXAMPLE 2: Risky Position")
print("=" * 80)
await monitor_margin_call_risk("S50Z25", contracts=10, current_equity=75000)

# Expected output shows different risk levels based on equity

2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:__init__:77 | TradingStatisticsService initialized with base_url=https://www.tfex.co.th


EXAMPLE 1: Healthy Position


2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:119 | Fetching TFEX trading statistics for S50Z25 from https://www.tfex.co.th/api/set/tfex/series/S50Z25/trading-statistics
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:fetch:303 | Fetch successful: url=https://www.tfex.co.th/api/set/tfex/series/S50Z25/trading-statistics, status=200, elapsed=0.01s, size=320 bytes
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:148 | Successfully fetched trading statistics for S50Z25: settlement_price=830.3, day_to_maturity=86, im=10080.0
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:__init__:77 | TradingStatisticsService initialized with base_url=https://www.tfex.co.th
2025-10-05 17:41:21 | INFO     | settfex

MARGIN CALL RISK MONITOR - S50Z25

Position: 10 contracts
Current Equity: 200,000.00 THB

MARGIN REQUIREMENTS:
  Initial Margin (IM): 100,800.00 THB
  Maintenance Margin (MM): 70,848.00 THB
  Margin Usage: 50.40%

EQUITY ANALYSIS:
  Equity above IM: +99,200.00 THB
  Equity above MM: +129,152.00 THB
  Equity vs MM: 282.29%

RISK ASSESSMENT:
  Risk Level: ✅ LOW
  Action: Healthy margin buffer. No action needed.

MAX SUSTAINABLE LOSS:
  Maximum loss: 129,152.00 THB
  Points cushion: 64.58 points
  Percentage cushion: 7.78%
  Margin call if price drops below: 765.72



EXAMPLE 2: Risky Position


2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:fetch:303 | Fetch successful: url=https://www.tfex.co.th/api/set/tfex/series/S50Z25/trading-statistics, status=200, elapsed=0.01s, size=320 bytes
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:148 | Successfully fetched trading statistics for S50Z25: settlement_price=830.3, day_to_maturity=86, im=10080.0


MARGIN CALL RISK MONITOR - S50Z25

Position: 10 contracts
Current Equity: 75,000.00 THB

MARGIN REQUIREMENTS:
  Initial Margin (IM): 100,800.00 THB
  Maintenance Margin (MM): 70,848.00 THB
  Margin Usage: 134.40%

EQUITY ANALYSIS:
  Equity above IM: -25,800.00 THB
  Equity above MM: +4,152.00 THB
  Equity vs MM: 105.86%

RISK ASSESSMENT:
  Risk Level: 🟠 CRITICAL
  Action: Add 25,800.00 THB to avoid margin call

MAX SUSTAINABLE LOSS:
  Maximum loss: 4,152.00 THB
  Points cushion: 2.08 points
  Percentage cushion: 0.25%
  Margin call if price drops below: 828.22


### Use Case 5: Multi-Contract Comparison

**Why compare contracts?**

- Different underlyings have different margin requirements
- Choose contracts that fit your capital and risk tolerance
- Understand leverage differences across products

Let's compare margin requirements across different TFEX products:

In [12]:
# Compare margin requirements across different products
async def compare_margin_requirements(symbols_list, capital=100000):
    """
    Compare margin requirements and position sizing across contracts.
    
    Args:
        symbols_list: List of TFEX symbols to compare
        capital: Available capital (THB)
    """
    print(f"MARGIN COMPARISON - Capital: {capital:,.0f} THB\n")
    print(f"{'Symbol':<12} {'IM':<12} {'MM':<12} {'Max Qty':<12} {'IM Usage':<12}")
    print("-" * 70)
    
    results = []
    
    for symbol in symbols_list:
        try:
            stats = await get_trading_statistics(symbol)
            
            # Calculate maximum contracts
            max_contracts = int(capital / stats.im)
            total_im = max_contracts * stats.im
            im_usage = (total_im / capital) * 100
            
            print(
                f"{stats.symbol:<12} "
                f"{stats.im:<12,.0f} "
                f"{stats.mm:<12,.0f} "
                f"{max_contracts:<12} "
                f"{im_usage:<12.1f}%"
            )
            
            results.append({
                'symbol': stats.symbol,
                'im': stats.im,
                'max_contracts': max_contracts
            })
            
        except Exception as e:
            print(f"{symbol:<12} Error: {e}")
    
    # Analysis
    if results:
        print("\nANALYSIS:")
        
        # Find lowest margin requirement
        lowest = min(results, key=lambda x: x['im'])
        highest = max(results, key=lambda x: x['im'])
        
        print(f"  Lowest margin: {lowest['symbol']} ({lowest['im']:,.0f} THB)")
        print(f"  Highest margin: {highest['symbol']} ({highest['im']:,.0f} THB)")
        print(f"  Margin range: {highest['im'] / lowest['im']:.2f}x difference")
        
        print(f"\n  With {capital:,.0f} THB, you can trade:")
        for result in results:
            print(f"    {result['max_contracts']} contracts of {result['symbol']}")

# Compare different TFEX products
# Note: Using S50 series as example (replace with actual diverse products in real use)
symbols_to_compare = ["S50V25", "S50X25", "S50Z25"]

await compare_margin_requirements(symbols_to_compare, capital=100000)

# Expected output:
# MARGIN COMPARISON - Capital: 100,000 THB
#
# Symbol       IM           MM           Max Qty      IM Usage    
# ----------------------------------------------------------------------
# S50V25       10,080       7,085        9            90.7%       
# S50X25       10,080       7,085        9            90.7%       
# S50Z25       10,080       7,085        9            90.7%

2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:__init__:77 | TradingStatisticsService initialized with base_url=https://www.tfex.co.th
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:119 | Fetching TFEX trading statistics for S50V25 from https://www.tfex.co.th/api/set/tfex/series/S50V25/trading-statistics
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:fetch:303 | Fetch successful: url=https://www.tfex.co.th/api/set/tfex/series/S50V25/trading-statistics, status=200, elapsed=0.01s, size=320 bytes
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:148 | Successfully fetched trading statistics for S50V25: settlement_price=834.0, day_to_maturity=26, im=10080.0
2025-10-05 17:41:21 | INFO     | settfex

MARGIN COMPARISON - Capital: 100,000 THB

Symbol       IM           MM           Max Qty      IM Usage    
----------------------------------------------------------------------
S50V25       10,080       7,085        9            90.7        %
S50X25       10,080       7,085        9            90.7        %
S50Z25       10,080       7,085        9            90.7        %

ANALYSIS:
  Lowest margin: S50V25 (10,080 THB)
  Highest margin: S50V25 (10,080 THB)
  Margin range: 1.00x difference

  With 100,000 THB, you can trade:
    9 contracts of S50V25
    9 contracts of S50X25
    9 contracts of S50Z25


## Error Handling

Always handle errors gracefully when working with live market data.

In [13]:
# Safe fetch with error handling
async def safe_fetch_statistics(symbol):
    """
    Fetch trading statistics with comprehensive error handling.
    
    Args:
        symbol: TFEX contract symbol
    
    Returns:
        TradingStatistics or None if failed
    """
    try:
        stats = await get_trading_statistics(symbol)
        print(f"✓ Successfully fetched statistics for {stats.symbol}")
        return stats
        
    except ConnectionError as e:
        print(f"✗ Network error for {symbol}: {e}")
        print("  Check your internet connection and try again.")
        return None
        
    except ValueError as e:
        print(f"✗ Invalid symbol {symbol}: {e}")
        print("  Verify the symbol exists using the series list service.")
        return None
        
    except Exception as e:
        print(f"✗ Unexpected error for {symbol}: {e}")
        print("  Please report this issue.")
        return None

# Test with valid symbol
print("Fetching valid symbol:")
stats = await safe_fetch_statistics("S50Z25")

if stats:
    print(f"  Settlement: {stats.settlement_price:.5f}")
    print(f"  Days to maturity: {stats.day_to_maturity}")

# Test with potentially invalid symbol
print("\nFetching potentially invalid symbol:")
stats = await safe_fetch_statistics("INVALID123")

# Expected output:
# Fetching valid symbol:
# ✓ Successfully fetched statistics for S50Z25
#   Settlement: 830.30000
#   Days to maturity: 85
#
# Fetching potentially invalid symbol:
# ✗ Invalid symbol INVALID123: ...
#   Verify the symbol exists using the series list service.

2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:__init__:77 | TradingStatisticsService initialized with base_url=https://www.tfex.co.th
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:119 | Fetching TFEX trading statistics for S50Z25 from https://www.tfex.co.th/api/set/tfex/series/S50Z25/trading-statistics
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:fetch:303 | Fetch successful: url=https://www.tfex.co.th/api/set/tfex/series/S50Z25/trading-statistics, status=200, elapsed=0.01s, size=320 bytes
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:148 | Successfully fetched trading statistics for S50Z25: settlement_price=830.3, day_to_maturity=86, im=10080.0
2025-10-05 17:41:21 | INFO     | settfex

Fetching valid symbol:
✓ Successfully fetched statistics for S50Z25
  Settlement: 830.30000
  Days to maturity: 86

Fetching potentially invalid symbol:


2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:fetch:303 | Fetch successful: url=https://www.tfex.co.th/api/set/tfex/series/INVALID123/trading-statistics, status=404, elapsed=0.05s, size=28 bytes


✗ Invalid symbol INVALID123: 12 validation errors for TradingStatistics
symbol
  Field required [type=missing, input_value={'message': 'Invalid Series'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
marketTime
  Field required [type=missing, input_value={'message': 'Invalid Series'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
lastTradingDate
  Field required [type=missing, input_value={'message': 'Invalid Series'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
dayToMaturity
  Field required [type=missing, input_value={'message': 'Invalid Series'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
settlementPattern
  Field required [type=missing, input_value={'message': 'Invalid Series'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
isOptions
  Field

In [14]:
# Verify symbol exists before fetching statistics
async def verify_and_fetch(symbol):
    """
    Verify symbol exists in series list before fetching statistics.
    
    Args:
        symbol: TFEX contract symbol
    
    Returns:
        TradingStatistics or None
    """
    print(f"Verifying {symbol}...")
    
    # Get series list
    series_list = await get_series_list()
    
    # Check if symbol exists
    series = series_list.get_symbol(symbol)
    
    if not series:
        print(f"✗ Symbol {symbol} not found in TFEX series list")
        print("\n  Tip: Search for similar symbols:")
        
        # Find similar symbols
        prefix = symbol[:3]
        similar = [s.symbol for s in series_list.series if s.symbol.startswith(prefix)][:5]
        
        if similar:
            print(f"  Found symbols starting with '{prefix}': {', '.join(similar)}")
        
        return None
    
    # Check if active
    if not series.active:
        print(f"⚠️  Warning: {symbol} exists but is NOT ACTIVE")
        print(f"  Expiry: {series.last_trading_date.date()}")
        print("  Consider using an active contract instead.")
    
    # Fetch statistics
    print(f"✓ Symbol verified. Fetching statistics...")
    stats = await get_trading_statistics(symbol)
    
    return stats

# Test verification
stats = await verify_and_fetch("S50Z25")

if stats:
    print(f"\n✓ Success! Settlement: {stats.settlement_price:.5f}")

# Expected output:
# Verifying S50Z25...
# ✓ Symbol verified. Fetching statistics...
#
# ✓ Success! Settlement: 830.30000

2025-10-05 17:41:21 | INFO     | settfex.services.tfex.list:__init__:177 | TFEXSeriesListService initialized with base_url=https://www.tfex.co.th
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.list:fetch_series_list:198 | Fetching TFEX series list from https://www.tfex.co.th/api/set/tfex/series/list
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:fetch:303 | Fetch successful: url=https://www.tfex.co.th/api/set/tfex/series/list, status=200, elapsed=0.02s, size=252286 bytes
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.list:fetch_series_list:227 | Successfully fetched 708 TFEX series from TFEX API (708 active, 576 futures, 132 options)
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:__init__:77 | TradingStatisticsService initialized with base_url=https://www.tfex.co.th
2025-10-05

Verifying S50Z25...
✓ Symbol verified. Fetching statistics...


2025-10-05 17:41:21 | INFO     | settfex.utils.data_fetcher:fetch:303 | Fetch successful: url=https://www.tfex.co.th/api/set/tfex/series/S50Z25/trading-statistics, status=200, elapsed=0.01s, size=320 bytes
2025-10-05 17:41:21 | INFO     | settfex.services.tfex.trading_statistics:fetch_trading_statistics:148 | Successfully fetched trading statistics for S50Z25: settlement_price=830.3, day_to_maturity=86, im=10080.0



✓ Success! Settlement: 830.30000


## Next Steps

Congratulations! You now know how to work with TFEX trading statistics for professional derivatives trading.

### Related Notebooks
- **01_series_list.ipynb** - Learn how to discover and filter available TFEX contracts

### Documentation
- [TFEX Trading Statistics Docs](../../docs/settfex/services/tfex/trading_statistics.md) - Complete API reference
- [TFEX Series List Docs](../../docs/settfex/services/tfex/list.md) - Contract discovery and filtering

### Key Takeaways

1. **Settlement prices** determine daily mark-to-market P/L
2. **Initial Margin (IM)** is required to open positions
3. **Maintenance Margin (MM)** is the minimum to avoid margin calls
4. **Days to maturity** guides rollover timing (roll 5-10 days early)
5. **Position sizing** based on margin = contracts × IM / capital
6. **Daily P/L** = price_change × multiplier × contracts
7. **Margin calls** occur when equity drops below MM
8. **Roll cost** = next_month_price - front_month_price

### Margin Formula Summary

```
Max Contracts = Available Capital / Initial Margin (IM)
Total Margin = Contracts × IM
Margin Call If: Account Equity < (Contracts × MM)
Daily P/L = (Today Settlement - Prior Settlement) × Multiplier × Contracts
Roll Cost = (Next Month Settlement - Front Month Settlement) × Multiplier
```

### Risk Management Tips

1. **Never use 100% of capital** - Leave buffer for adverse moves
2. **Monitor margin daily** - Avoid surprise margin calls
3. **Roll early** - Don't wait until last few days before expiration
4. **Understand leverage** - Small price moves = large P/L impact
5. **Use stop losses** - Protect against large losses

### Real-World Applications

- **Trading systems** - Automated position sizing and risk management
- **Portfolio analytics** - Track margin usage and risk exposure
- **Alert systems** - Notifications for margin calls or expiration
- **Backtesting** - Historical settlement prices for strategy testing

Happy trading! 📊